In [147]:
import pandas as pd

In [148]:
df=pd.read_csv('spam.csv')
df.head(2)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [149]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else  0)

In [150]:
df.drop(['Category'],axis=1,inplace=True)

In [151]:
df.head()

,Message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [152]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.Message,df['spam'],test_size=0.2)

In [153]:
len(x_test)


1115

In [154]:
from sklearn.feature_extraction.text import CountVectorizer
v=CountVectorizer()
x_train_count=v.fit_transform(x_train.values)
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [155]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train_count,y_train)


MultinomialNB()

In [156]:
# Transform test data using the vectorizer fitted on training data
x_test_count = v.transform(x_test)
model.score(x_test_count, y_test)

0.9865470852017937

In [157]:
emails = [
    "Congratulations! You have won a $1000 gift card. Click here to claim now.",
    "Hi, can we meet tomorrow for the project discussion?",
    "Urgent: Your account will be suspended. Verify your password now.",
    "Your Amazon order has been shipped. Expected delivery: Tuesday."
]
# transform the emails into features
emails_count=v.transform(emails)
predictions=model.predict(emails_count)

for email, label in zip(emails, predictions):
    print(f"Email: {email}\nPrediction: {'Spam' if label==1 else 'Not Spam'}\n")

Email: Congratulations! You have won a $1000 gift card. Click here to claim now.
Prediction: Spam

Email: Hi, can we meet tomorrow for the project discussion?
Prediction: Not Spam

Email: Urgent: Your account will be suspended. Verify your password now.
Prediction: Spam

Email: Your Amazon order has been shipped. Expected delivery: Tuesday.
Prediction: Spam



In [158]:
# save this model
import pickle
with open('spam_checker.pkl','wb') as f:
    pickle.dump(model,f)

In [159]:
# save vectorizer(transformatoin)
with open("transformer.pkl", "wb") as f:
    pickle.dump(v, f)

In [165]:
txt='Our records show you overpaid for (a product or service). Kindly supply your bank routing and account number to receive your refund.'
# txt='''Never used PhonePe? Get FLAT ₹100 CASHBACK on Vi SuperHero/NonStop Hero! Recharge via PhonePe UPI. T&C. https://phon.pe/viwb'''
# txt="Share stories of our Tiranga. Hoist the flag and assist others. Upload your Tiranga Selfies! https://mybharat.gov.in/mega_events/har-ghar-tiranga-2025"
txt_count=v.transform([txt])
predictions=model.predict(txt_count)
print(f"Message: {txt} \nPrediction : {'Spam' if 1==predictions else 'Not Spam'}")

Message: Our records show you overpaid for (a product or service). Kindly supply your bank routing and account number to receive your refund. 
Prediction : Spam


In [166]:
# Second way using pipeline
from sklearn.pipeline import Pipeline
clf=Pipeline([
   ( 'vectorizer',CountVectorizer()),
   ('nb',MultinomialNB())
])

In [167]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [170]:
txt='Our records show you overpaid for (a product or service). Kindly supply your bank routing and account number to receive your refund'
pred=clf.predict([txt])
print(f"Message: {txt}\nPrediction:{'spam' if pred==1 else 'not spam'}")

Message: Our records show you overpaid for (a product or service). Kindly supply your bank routing and account number to receive your refund
Prediction:spam


In [171]:
# save this pipeline model
with open ('spam-checker-pipeline.pkl','wb') as f:
    pickle.dump(clf,f)